In [ ]:
import progressbar
import h5py
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import pytz
import warnings
import pylab as plt
%matplotlib inline
from matplotlib.colors import LogNorm

import search_dispersion_events 
import importlib
importlib.reload(search_dispersion_events)

In [ ]:
df = pd.read_csv('data/train.csv', parse_dates=['start_time', 'end_time'])
df.head()

In [ ]:
def get_metrics(row):
    importlib.reload(search_dispersion_events)
    
    # Do computation --------------------------------------------------
    fh = search_dispersion_events.read_file(row.filename)

    dEicdt_smooth, Eic = search_dispersion_events.estimate_log_Eic_smooth_derivative(fh)

    _, integrand,integral, upper_area_frac = search_dispersion_events.walk_and_integrate(
        fh, dEicdt_smooth, Eic, search_dispersion_events.DEFAULT_INTERVAL_LENGTH,
        return_integrand=True
    )
        
    i = fh['t'].searchsorted(row.start_time)
    j = fh['t'].searchsorted(row.end_time)

    return row.name, integral[i:j].max(), upper_area_frac[i:j].max(), fh['mlat'][i], fh['mlat'][j]

In [ ]:
from joblib import Parallel, delayed

disp_df = df[df['class'] == 1]
metric_rows = Parallel(n_jobs=12)(delayed(get_metrics)(row) for _, row in disp_df.iterrows())
metric_df = pd.DataFrame(metric_rows, columns=['row_name', 'integral', 'upper_area_frac', 'mlat_start', 'mlat_end'])

In [ ]:
plt.hist(metric_df.integral, bins=40)
plt.xlabel('Integral')
plt.ylabel('Bin Count')
plt.title('Max Integral in Sub-Windows for Example Set')
None

In [ ]:
plt.hist(metric_df.upper_area_frac, bins=40)
plt.xlabel('Upper Area Fraction')
plt.ylabel('Bin Count')
plt.title('Max Upper Area Fraction in Sub-Windows for Example Set')

None